In [ ]:
# 1 min data pitch of the SP500 didnt get anywhere
# lets try bitcoin vs gold vs SP500

In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import os
import datetime
from datetime import datetime, time, timedelta, timezone
from concurrent import futures
from pandas import DataFrame
import pandas_datareader.data as web
import pymongo
import requests_cache
import time
import pytz

In [2]:
# Original start date in Eastern Time (UTC-5)
start_date_et = datetime(2022, 11, 10, 10, 0, tzinfo=timezone(timedelta(hours=-5)))
end_date_et = datetime(2023, 11, 2, 3, 0, tzinfo=timezone(timedelta(hours=-5)))
start_date_btc = datetime(2022, 11, 8, 10, 0, tzinfo=timezone(timedelta(hours=-5))) #i'll trim it later. its a whole thing.

# Convert to UTC
start_date_utc = start_date_et.astimezone(timezone.utc)
end_date_utc = end_date_et.astimezone(timezone.utc)

print(start_date_et)
print(end_date_et)

print(start_date_utc)
print(end_date_utc)

2022-11-10 10:00:00-05:00
2023-11-02 03:00:00-05:00
2022-11-10 15:00:00+00:00
2023-11-02 08:00:00+00:00


In [3]:
# Indexes
btc = yf.Ticker("BTC-USD")
sp500 = yf.Ticker("^GSPC")
gold = yf.Ticker("GLD")  # or "GC=F" for futures

In [4]:
# Gold
gold_df = gold.history(start=start_date_et, end=end_date_et, interval="1h")# Fetch data for S&P 500 index

# SP500
sp500_df = sp500.history(start=start_date_et, end=end_date_et, interval="1h")# Fetch data for S&P 500 index

# BTC
btc_df = btc.history(start=start_date_btc, end=end_date_et, interval="1h")# Fetch data for S&P 500 index

In [5]:
# Convert gold_df and sp500_df to UTC
gold_df.index = gold_df.index.tz_convert('UTC')
sp500_df.index = sp500_df.index.tz_convert('UTC')

In [6]:
# Convert datetime indices to timestamp columns
gold_df.reset_index(inplace=True)
gold_df.rename(columns={'Datetime': 'Timestamp'}, inplace=True)

sp500_df.reset_index(inplace=True)
sp500_df.rename(columns={'Datetime': 'Timestamp'}, inplace=True)

btc_df.reset_index(inplace=True)
btc_df.rename(columns={'Datetime': 'Timestamp'}, inplace=True)

In [7]:
# Duplicate and adjust btc_df timestamps
btc_df['real Timestamp'] = btc_df['Timestamp']
btc_original_df = btc_df.copy()
btc_df['Timestamp'] = btc_df['Timestamp'] + pd.Timedelta(minutes=30)

# the BTC columns are now 30 minute ahead of where they really are
# real time: 15:00 // time in df is 15:30
# this is not ideal.

In [8]:
sp500_df.head(2)

,Timestamp,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2022-11-10 14:30:00+00:00,3859.889893,3912.139893,3859.889893,3905.110107,0,0.0,0.0
1,2022-11-10 15:30:00+00:00,3905.070068,3915.699951,3887.709961,3914.969971,417625037,0.0,0.0


In [9]:
btc_df.head(2)

,Timestamp,Open,High,Low,Close,Volume,Dividends,Stock Splits,real Timestamp
0,2022-11-08 15:30:00+00:00,19492.632812,19492.632812,19353.945312,19470.234375,0,0.0,0.0,2022-11-08 15:00:00+00:00
1,2022-11-08 16:30:00+00:00,19475.931641,20582.238281,19475.931641,20425.300781,9666437120,0.0,0.0,2022-11-08 16:00:00+00:00


In [10]:
sp500_df.tail(2)

,Timestamp,Open,High,Low,Close,Volume,Dividends,Stock Splits
1705,2023-11-01 18:30:00+00:00,4214.950195,4244.109863,4202.020020,4239.250000,412504000,0.0,0.0
1706,2023-11-01 19:30:00+00:00,4239.229980,4245.640137,4235.799805,4237.609863,430241000,0.0,0.0


In [11]:
# Convert the string timestamp to a datetime object
target_timestamp = pd.to_datetime('2022-11-10 14:30:00+00:00')

# Filter the DataFrame to keep only rows on or after the target timestamp
btc_df = btc_df[btc_df['Timestamp'] >= target_timestamp]

In [12]:
# Convert the string timestamp to a datetime object
target_timestamp = pd.to_datetime('2023-11-01 19:30:00+00:00')

# Filter the DataFrame to keep only rows on or after the target timestamp
btc_df = btc_df[btc_df['Timestamp'] <= target_timestamp]

In [13]:
btc_df.head(2)

,Timestamp,Open,High,Low,Close,Volume,Dividends,Stock Splits,real Timestamp
47,2022-11-10 14:30:00+00:00,17572.923828,17606.298828,17434.164062,17603.916016,944775168,0.0,0.0,2022-11-10 14:00:00+00:00
48,2022-11-10 15:30:00+00:00,17606.634766,17819.566406,17445.076172,17489.992188,0,0.0,0.0,2022-11-10 15:00:00+00:00


In [14]:
btc_endstrimmed_df = btc_df.copy()

In [15]:
btc_df.reset_index(drop=True, inplace=True)

In [16]:
## ok. start and end are now unified 

# 9:30 am eastern is 2:30 / 14:30 UTC
# 4pm eastern is 9:00 / 21:00 UTC
# 11/10/22 was a thursday

In [17]:
gold_df.head(10)

,Timestamp,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
0,2022-11-10 14:30:00+00:00,162.009995,163.190002,161.654999,162.994995,0,0.0,0.0,0.0
1,2022-11-10 15:30:00+00:00,163.000000,163.220001,162.309998,162.699997,2247057,0.0,0.0,0.0
2,2022-11-10 16:30:00+00:00,162.699997,163.009995,162.679993,162.869995,749521,0.0,0.0,0.0
3,2022-11-10 17:30:00+00:00,162.875000,162.990005,162.559998,162.949997,578755,0.0,0.0,0.0
4,2022-11-10 18:30:00+00:00,162.949997,163.110001,162.889999,163.000000,673003,0.0,0.0,0.0
5,2022-11-10 19:30:00+00:00,163.000000,163.339996,162.979996,163.210007,518888,0.0,0.0,0.0
6,2022-11-10 20:30:00+00:00,163.214996,163.509995,163.149994,163.460007,1053560,0.0,0.0,0.0
7,2022-11-11 14:30:00+00:00,163.914001,164.240005,163.419998,164.199997,1243430,0.0,0.0,0.0
8,2022-11-11 15:30:00+00:00,164.169998,164.279999,163.839996,163.839996,850611,0.0,0.0,0.0
9,2022-11-11 16:30:00+00:00,163.839996,164.309998,163.800003,164.279999,496261,0.0,0.0,0.0


In [18]:
btc_df.head(10)

,Timestamp,Open,High,Low,Close,Volume,Dividends,Stock Splits,real Timestamp
0,2022-11-10 14:30:00+00:00,17572.923828,17606.298828,17434.164062,17603.916016,944775168,0.0,0.0,2022-11-10 14:00:00+00:00
1,2022-11-10 15:30:00+00:00,17606.634766,17819.566406,17445.076172,17489.992188,0,0.0,0.0,2022-11-10 15:00:00+00:00
2,2022-11-10 16:30:00+00:00,17497.273438,17872.724609,17484.052734,17797.347656,0,0.0,0.0,2022-11-10 16:00:00+00:00
3,2022-11-10 17:30:00+00:00,17790.031250,17790.031250,17202.929688,17202.929688,0,0.0,0.0,2022-11-10 17:00:00+00:00
4,2022-11-10 18:30:00+00:00,17208.423828,17427.103516,17208.423828,17343.730469,0,0.0,0.0,2022-11-10 18:00:00+00:00
5,2022-11-10 19:30:00+00:00,17342.998047,17445.505859,17285.185547,17445.505859,0,0.0,0.0,2022-11-10 19:00:00+00:00
6,2022-11-10 20:30:00+00:00,17454.019531,18022.648438,17391.619141,17989.996094,0,0.0,0.0,2022-11-10 20:00:00+00:00
7,2022-11-10 21:30:00+00:00,18001.970703,18054.312500,17803.728516,17808.710938,0,0.0,0.0,2022-11-10 21:00:00+00:00
8,2022-11-10 22:30:00+00:00,17808.710938,17850.578125,17730.615234,17835.083984,0,0.0,0.0,2022-11-10 22:00:00+00:00
9,2022-11-10 23:30:00+00:00,17831.343750,17836.314453,17541.423828,17587.781250,0,0.0,0.0,2022-11-10 23:00:00+00:00


In [19]:
btc_df = btc_df[btc_df['Timestamp'].isin(gold_df['Timestamp'])]

In [20]:
btc_df.reset_index(drop=True, inplace=True)
gold_df.reset_index(drop=True, inplace=True)

In [21]:
sp500_df = sp500_df[sp500_df['Timestamp'].isin(gold_df['Timestamp'])]
sp500_df.reset_index(drop=True, inplace=True)

In [22]:
# Print the start and end times for each dataframe
print("BTC DataFrame Start:", btc_df.index.min(), "End:", btc_df.index.max())
print("Gold DataFrame Start:", gold_df.index.min(), "End:", gold_df.index.max())
print("S&P 500 DataFrame Start:", sp500_df.index.min(), "End:", sp500_df.index.max())

# Print the number of rows
print("BTC DataFrame Rows:", len(btc_df))
print("Gold DataFrame Rows:", len(gold_df))
print("S&P 500 DataFrame Rows:", len(sp500_df))

BTC DataFrame Start: 0 End: 1701
Gold DataFrame Start: 0 End: 1706
S&P 500 DataFrame Start: 0 End: 1706
BTC DataFrame Rows: 1702
Gold DataFrame Rows: 1707
S&P 500 DataFrame Rows: 1707


# btc is missing data for 3/13/23 from 19:30
# 3/14/23 for 13:30/14:30/15:30
# 10/23/23 at 15:30
# i'm just deleting it all from gold and sp500 to get this done

In [23]:
sp500_df = sp500_df[sp500_df['Timestamp'].isin(btc_df['Timestamp'])]
sp500_df.reset_index(drop=True, inplace=True)
gold_df = gold_df[gold_df['Timestamp'].isin(btc_df['Timestamp'])]
gold_df.reset_index(drop=True, inplace=True)

In [24]:
timestamp_column_dtype = gold_df['Timestamp'].dtype
print(f"gold {timestamp_column_dtype}")
timestamp_column_dtype = sp500_df['Timestamp'].dtype
print(f"sp500 {timestamp_column_dtype}")
timestamp_column_dtype = btc_df['Timestamp'].dtype
print(f"btc {timestamp_column_dtype}")

gold datetime64[ns, UTC]
sp500 datetime64[ns, UTC]
btc datetime64[ns, UTC]


In [25]:
btc_match = btc_df.copy()
gold_match = gold_df.copy()
sp500_match = sp500_df.copy()

In [26]:
gold_match = gold_match.drop(columns=['Capital Gains', 'Stock Splits', 'Dividends'])
sp500_match = sp500_match.drop(columns=['Dividends', 'Stock Splits'])
btc_match = btc_match.drop(columns=['Dividends', 'Stock Splits', 'real Timestamp'])

In [27]:
gold_match['Ticker'] = 'GLD'
sp500_match['Ticker'] = '^GSPC'
btc_match['Ticker'] = 'BTC-USD'

In [ ]:
btc_match.head(10)

In [ ]:
sp500_match.head(2)

In [ ]:
gold_match.head(2)

In [28]:
# Print the start and end times for each dataframe
print("BTC DataFrame Start:", btc_match.index.min(), "End:", btc_match.index.max())
print("Gold DataFrame Start:", gold_match.index.min(), "End:", gold_match.index.max())
print("S&P 500 DataFrame Start:", sp500_match.index.min(), "End:", sp500_match.index.max())

# Print the number of rows
print("BTC DataFrame Rows:", len(btc_match))
print("Gold DataFrame Rows:", len(gold_match))
print("S&P 500 DataFrame Rows:", len(sp500_match))

BTC DataFrame Start: 0 End: 1701
Gold DataFrame Start: 0 End: 1701
S&P 500 DataFrame Start: 0 End: 1701
BTC DataFrame Rows: 1702
Gold DataFrame Rows: 1702
S&P 500 DataFrame Rows: 1702


In [29]:
# Function to add rolling averages to a dataframe
def add_rolling_averages(df, hours_per_day=24):
    for window_size, label in [(3, '3h'), (6, '6h'), (1*hours_per_day, '1d'), 
                               (3*hours_per_day, '3d'), (7*hours_per_day, '1w')]:
        for column in ['Close', 'Volume']:
            rolling_column_name = f'{column}_rolling_{label}'
            df[rolling_column_name] = df[column].rolling(window=window_size).mean()
    return df

In [31]:
gold_match = add_rolling_averages(gold_match)
sp500_match = add_rolling_averages(sp500_match)
btc_match = add_rolling_averages(btc_match)

In [32]:
columns_list = gold_match.columns.tolist()
print(columns_list)

['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume', 'Ticker', 'Close_rolling_3h', 'Volume_rolling_3h', 'Close_rolling_6h', 'Volume_rolling_6h', 'Close_rolling_1d', 'Volume_rolling_1d', 'Close_rolling_3d', 'Volume_rolling_3d', 'Close_rolling_1w', 'Volume_rolling_1w']


In [ ]:
#drops open/high/low
#gold_match = gold_match[['Timestamp', 'Close', 'Volume', 'Ticker', 'Close_rolling_3h', 'Close_rolling_6h', 'Close_rolling_1d', 'Close_rolling_3d', 'Close_rolling_1w']]
#sp500_match = sp500_match[['Timestamp', 'Close', 'Volume', 'Ticker', 'Close_rolling_3h', 'Close_rolling_6h', 'Close_rolling_1d', 'Close_rolling_3d', 'Close_rolling_1w']]
#btc_match = btc_match[['Timestamp', 'Close', 'Volume', 'Ticker', 'Close_rolling_3h', 'Close_rolling_6h', 'Close_rolling_1d', 'Close_rolling_3d', 'Close_rolling_1w']]

In [ ]:
# merge it into one table

In [33]:
# Convert the 'Timestamp' column to datetime if it's not already
gold_match['Timestamp'] = pd.to_datetime(gold_match['Timestamp'])
sp500_match['Timestamp'] = pd.to_datetime(sp500_match['Timestamp'])
btc_match['Timestamp'] = pd.to_datetime(btc_match['Timestamp'])

In [34]:
# Set the 'Timestamp' column as the index
gold_match.set_index('Timestamp', inplace=True)
sp500_match.set_index('Timestamp', inplace=True)
btc_match.set_index('Timestamp', inplace=True)

In [35]:
gold_match.head(10)

,Open,High,Low,Close,Volume,Ticker,Close_rolling_3h,Volume_rolling_3h,Close_rolling_6h,Volume_rolling_6h,Close_rolling_1d,Volume_rolling_1d,Close_rolling_3d,Volume_rolling_3d,Close_rolling_1w,Volume_rolling_1w
Timestamp,,,,,,,,,,,,,,,,
2022-11-10 14:30:00+00:00,162.009995,163.190002,161.654999,162.994995,0,GLD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-10 15:30:00+00:00,163.000000,163.220001,162.309998,162.699997,2247057,GLD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-10 16:30:00+00:00,162.699997,163.009995,162.679993,162.869995,749521,GLD,162.854996,9.988593e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-10 17:30:00+00:00,162.875000,162.990005,162.559998,162.949997,578755,GLD,162.839996,1.191778e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-10 18:30:00+00:00,162.949997,163.110001,162.889999,163.000000,673003,GLD,162.939997,6.670930e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-10 19:30:00+00:00,163.000000,163.339996,162.979996,163.210007,518888,GLD,163.053335,5.902153e+05,162.954165,794537.333333,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-10 20:30:00+00:00,163.214996,163.509995,163.149994,163.460007,1053560,GLD,163.223338,7.484837e+05,163.031667,970130.666667,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-11 14:30:00+00:00,163.914001,164.240005,163.419998,164.199997,1243430,GLD,163.623337,9.386260e+05,163.281667,802859.500000,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-11 15:30:00+00:00,164.169998,164.279999,163.839996,163.839996,850611,GLD,163.833333,1.049200e+06,163.443334,819707.833333,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
sp500_match.head(10)

In [ ]:
btc_match.head(10)

In [37]:
# Merge the data frames on the 'Timestamp' index
# Start by merging gold and S&P 500 data frames
merged_df = pd.merge(gold_match, sp500_match, left_index=True, right_index=True, how='outer', suffixes=('_gold', '_sp500'))

In [38]:
# Then merge the resulting data frame with the BTC data frame
final_merged_df = pd.merge(merged_df, btc_match, left_index=True, right_index=True, how='outer')

In [39]:
final_merged_df.head(5)

,Open_gold,High_gold,Low_gold,Close_gold,Volume_gold,Ticker_gold,Close_rolling_3h_gold,Volume_rolling_3h_gold,Close_rolling_6h_gold,Volume_rolling_6h_gold,...,Close_rolling_3h,Volume_rolling_3h,Close_rolling_6h,Volume_rolling_6h,Close_rolling_1d,Volume_rolling_1d,Close_rolling_3d,Volume_rolling_3d,Close_rolling_1w,Volume_rolling_1w
Timestamp,,,,,,,,,,,,,,,,,,,,,
2022-11-10 14:30:00+00:00,162.009995,163.190002,161.654999,162.994995,0,GLD,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-10 15:30:00+00:00,163.000000,163.220001,162.309998,162.699997,2247057,GLD,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-10 16:30:00+00:00,162.699997,163.009995,162.679993,162.869995,749521,GLD,162.854996,9.988593e+05,NaN,NaN,...,17630.418620,314925056.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-10 17:30:00+00:00,162.875000,162.990005,162.559998,162.949997,578755,GLD,162.839996,1.191778e+06,NaN,NaN,...,17496.756510,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-10 18:30:00+00:00,162.949997,163.110001,162.889999,163.000000,673003,GLD,162.939997,6.670930e+05,NaN,NaN,...,17448.002604,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Now, final_merged_df contains all data. If there are any NaN values from non-overlapping timestamps, you can decide how to handle them.
# For example, to forward-fill NaN values:
#final_merged_df.fillna(method='ffill', inplace=True)

In [40]:
# If you want to back-fill instead:
# final_merged_df.fillna(method='bfill', inplace=True)

# If you want to drop any rows that still have NaN values:
# final_merged_df.dropna(inplace=True)

# If you want to reset the index so 'Timestamp' becomes a column again:
final_merged_df.reset_index(inplace=True)

In [41]:
# Show the first few rows of the final merged data frame
print(final_merged_df.head())

                  Timestamp   Open_gold   High_gold    Low_gold  Close_gold  \
0 2022-11-10 14:30:00+00:00  162.009995  163.190002  161.654999  162.994995   
1 2022-11-10 15:30:00+00:00  163.000000  163.220001  162.309998  162.699997   
2 2022-11-10 16:30:00+00:00  162.699997  163.009995  162.679993  162.869995   
3 2022-11-10 17:30:00+00:00  162.875000  162.990005  162.559998  162.949997   
4 2022-11-10 18:30:00+00:00  162.949997  163.110001  162.889999  163.000000   

   Volume_gold Ticker_gold  Close_rolling_3h_gold  Volume_rolling_3h_gold  \
0            0         GLD                    NaN                     NaN   
1      2247057         GLD                    NaN                     NaN   
2       749521         GLD             162.854996            9.988593e+05   
3       578755         GLD             162.839996            1.191778e+06   
4       673003         GLD             162.939997            6.670930e+05   

   Close_rolling_6h_gold  ...  Close_rolling_3h  Volume_rollin

In [42]:
# Print the start and end times for each dataframe
print("Merged DataFrame Start:", final_merged_df.index.min(), "End:", final_merged_df.index.max())

# Print the number of rows
print("Merged DataFrame Rows:", len(final_merged_df))

Merged DataFrame Start: 0 End: 1701
Merged DataFrame Rows: 1702


In [43]:
columns_list = final_merged_df.columns.tolist()
print(columns_list)

['Timestamp', 'Open_gold', 'High_gold', 'Low_gold', 'Close_gold', 'Volume_gold', 'Ticker_gold', 'Close_rolling_3h_gold', 'Volume_rolling_3h_gold', 'Close_rolling_6h_gold', 'Volume_rolling_6h_gold', 'Close_rolling_1d_gold', 'Volume_rolling_1d_gold', 'Close_rolling_3d_gold', 'Volume_rolling_3d_gold', 'Close_rolling_1w_gold', 'Volume_rolling_1w_gold', 'Open_sp500', 'High_sp500', 'Low_sp500', 'Close_sp500', 'Volume_sp500', 'Ticker_sp500', 'Close_rolling_3h_sp500', 'Volume_rolling_3h_sp500', 'Close_rolling_6h_sp500', 'Volume_rolling_6h_sp500', 'Close_rolling_1d_sp500', 'Volume_rolling_1d_sp500', 'Close_rolling_3d_sp500', 'Volume_rolling_3d_sp500', 'Close_rolling_1w_sp500', 'Volume_rolling_1w_sp500', 'Open', 'High', 'Low', 'Close', 'Volume', 'Ticker', 'Close_rolling_3h', 'Volume_rolling_3h', 'Close_rolling_6h', 'Volume_rolling_6h', 'Close_rolling_1d', 'Volume_rolling_1d', 'Close_rolling_3d', 'Volume_rolling_3d', 'Close_rolling_1w', 'Volume_rolling_1w']


In [44]:
final_merged_df.to_csv('gold_sp500_btc.csv', index=False)
